# SignXAI2 with TensorFlow - Basic Usage

This tutorial demonstrates TensorFlow/Keras models with dynamic method parsing.

## Key Features:
- Dynamic method parsing
- Unified API
- No wrapper functions

In [ ]:
import tensorflow as tf
import numpy as np
from signxai.api import explain
from signxai.utils.utils import remove_softmax

In [ ]:
# Load a pre-trained model
from tensorflow.keras.applications import VGG16
model = VGG16(weights='imagenet')

# Dynamic method examples
methods = [
    'gradient',
    'gradient_x_input',
    'smoothgrad_noise_0_3_samples_50',
    'integrated_gradients_steps_100',
    'lrp_epsilon_0_25'
]

# Generate explanations
for method in methods:
    explanation = explain(model, input_image, method_name=method)